In [17]:
import pandas as pd
ts1 = pd.read_json("../data/a7fb4f0f659a32eb35406c20111f5047.data", orient='index')
ts2 = pd.read_json("../data/adbd497977188ee19a91789a1aa74de6.data", orient='index')


In [49]:
s1=list(ts1[ts1["siren"] == 315620104]["search"])
print(s1)

[['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.europages.fr/ETABLISSEMENTS-BORIE/FRA518096-00101.html', 'https://societeinfo.com/app/recherche/societe/315620104']]


In [44]:
s2=ts2[ts2["siren"] == 315620104]["search"]
print(s2)

0    ['https://www.dnb.com/business-directory/compa...
Name: search, dtype: object


In [45]:
for i in s1:
    print(i)
    for ii in i:
        print(ii)

['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.europages.fr/ETABLISSEMENTS-BORIE/FRA518096-00101.html', 'https://societeinfo.com/app/recherche/societe/315620104']
https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html
https://www.societe.com/societe/etablissements-borie-31

In [46]:
for i in s2:
    print(i)
    for ii in i:
        print(ii)

['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.europages.fr/ETABLISSEMENTS-BORIE/FRA518096-00101.html', 'https://societeinfo.com/app/recherche/societe/315620104']
[
'
h
t
t
p
s
:
/
/
w
w
w
.
d
n
b
.
c
o
m
/
b
u
s
i
n
e
s
s
-
d
i
r
e
c
t
o
r
y
/
c
o
m
p
a
n
y
-
p
r
o
f
i
l
e
s
.
e
t
a
b
l
i
s
s
e
m
e
n
t
s
_
b
o
r
i
e
.
4
a
4
4
7
a


In [57]:
bufferSearch = pd.read_csv("../data/bufferSearch.csv")
from ast import literal_eval
bufferSearch['search'] = df['search'].apply(literal_eval)

buf=bufferSearch.loc[bufferSearch['denomination'] == "ETABLISSEMENTS BORIE", 'search']
r=buf.iloc[0]
print (r)
type(r)

ValueError: malformed node or string: ['https://www.dnb.com/business-directory/company-profiles.provost_freres.66acf47c517a112c017d56760ecf141f.html', 'https://www.bloomberg.com/profile/company/4753169Z:FP', 'https://www.mytropicaltimber.org/en/c/1/agencies/284/provost-freres', 'https://scierie-provost.fr/', 'https://www.yelp.com/biz/provost-freres-sauz%C3%A9-vaussais-2', 'https://www.societe.com/societe/provost-freres-026280206.html', 'http://www.boistropicaux.org/fournisseurs/284/provost-freres', 'https://www.pagesjaunes.fr/pros/00438034', 'https://october.eu/project/provost-freres/', 'https://us.kompass.com/c/provost-freres/fr0170497/', 'https://www.usinenouvelle.com/article/la-scierie-provost-freres-investit-1-7-million-d-euros-pour-moderniser-son-site-des-deux-sevres.N914694']

In [55]:
from googlesearch import search
r=search("ETABLISSEMENTS BORIE")
print(r)
type(r)

['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://societeinfo.com/app/recherche/societe/315620104', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.manageo.fr/entreprises/315620104.html']


list

In [41]:
for idx, word in enumerate(s):
    print (idx, word)

0 ['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.europages.fr/ETABLISSEMENTS-BORIE/FRA518096-00101.html', 'https://societeinfo.com/app/recherche/societe/315620104']


In [11]:
ls=list(s)
ls

["['https://www.dnb.com/business-directory/company-profiles.etablissements_borie.4a447af8972e61a341141bc765243fd7.html', 'https://www.societe.com/societe/etablissements-borie-315620104.html', 'https://www.societe.com/etablissement/etablissements-borie-31562010400036.html', 'https://gb.kompass.com/c/etablissements-borie/fr1920791/', 'https://www.verif.com/societe/ETABLISSEMENTS-BORIE-315620104/', 'https://bo.linkedin.com/company/etablissements-borie', 'http://entreprises.lefigaro.fr/etablissements-borie-43/entreprise-315620104', 'http://www.scierie-borie.fr/wp-content/uploads/2015/10/PEFC.pdf', 'https://www.infogreffe.com/entreprise-societe/315620104-etablissements-borie-430200B000830000.html', 'https://www.europages.fr/ETABLISSEMENTS-BORIE/FRA518096-00101.html', 'https://societeinfo.com/app/recherche/societe/315620104']"]

In [ ]:
for index, row in df.iterrows():
    if row["siren"] not in sirens:
        ld=ld+row["domains"]
        
    sirens.append(row["siren"])

In [46]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
df[["domains", "search"]]

,domains,search
0,"{1., g., t., E., o., p., d., u., w., 0., S., h...",['https://www.societe.com/societe/soc-exploita...
1,"{1., g., t., F., E., o., p., V., d., J., u., w...","['https://www.scieriedesblaches.fr/', 'https:/..."
2,"{1., g., t., E., o., p., u., w., 0., S., h., I...",['https://www.yelp.com/biz/clotures-piotin-oye...
3,"{1., g., t., E., o., p., q., d., u., w., 0., S...",['https://www.societe.com/societe/sarl-scierie...
4,"{1., g., t., E., o., p., V., d., u., w., 0., S...",['https://www.societe.com/societe/sarl-raboter...
5,"{1., g., t., o., p., d., u., w., S., 0., h., I...","['http://www.atcbois.fr/en/', 'http://www.atcb..."
6,"{1., g., t., E., o., p., q., d., u., w., 0., S...","['https://www.pagesjaunes.fr/pros/05615037', '..."


TypeError: count() takes exactly one argument (0 given)

In [44]:
import tldextract

def domains(row):
    ld = set()    
    for item in row["search"]:
        print (item)
        d = tldextract.extract(item)
        dom = '.'.join(d[1:3])
        ld.add(dom)
    return ld

In [12]:
import logging
import extract
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

params = {"codeCommuneEtablissement": "382*","etatAdministratifUniteLegale": "A",
'categorieJuridiqueUniteLegale': "5*",
'activitePrincipaleUniteLegale': "16.10A", 'etablissementSiege':True}
file =extract.query(logger, params)

import myGoogleSearch
ts=myGoogleSearch.go(file, logger)

2020-12-02 09:10:18,160 
QUERY!!!                       :
codeCommuneEtablissement       : 382*
etatAdministratifUniteLegale   : A
categorieJuridiqueUniteLegale  : 5*
activitePrincipaleUniteLegale  : 16.10A
etablissementSiege             : True
2020-12-02 09:10:18,163 Available (../data/85d61063e22801b44c990a116e3d88c4.data)
2020-12-02 09:10:18,187 on recalcule groupEffectif
2020-12-02 09:10:18,204 - - - - - - -
Nb entreprises                : 6 
Fichier                       : ../data/85d61063e22801b44c990a116e3d88c4.data
2020-12-02 09:10:18,214 IN --> go ../data/85d61063e22801b44c990a116e3d88c4.data
2020-12-02 09:10:18,233 Data ready
2020-12-02 09:10:18,237  0 / 6 (to be searched / total)
2020-12-02 09:10:18,247 Begin calcOwnedDomains
2020-12-02 09:10:18,251 Domains to remove ['societe.com', 'kompass.com', 'lefigaro.fr', 'verif.com', 'pagesjaunes.fr', 'infogreffe.com']
2020-12-02 09:10:18,254 SCIERIE DES BLACHES : VOTRE ARTISAN SCIEUR IMPLANTÉ À MARCILLOLES vs SCIERIE DES BLACHES : 0

20 à 49 salariés
10 à 19 salariés
3 à 5 salariés
1 ou 2 salariés
1 ou 2 salariés
3 à 5 salariés


2020-12-02 09:10:19,652 Done.


In [11]:
ts.columns

Index(['siren', 'siret', 'activite', 'activiteLong', 'NAF 732',
       'INTITULÉS 732', 'categorie', 'effectifBrut', 'commune', 'catJuridique',
       'denomination', 'departement', 'CA 1', 'CA 2', 'CA 3',
       'etatAdministratif', 'CA_average', 'groupCA', 'groupEffectif',
       'effectif', 'region', 'search', 'isSearched', 'domains', 'ownedDomain',
       'ownedDomain2', 'searchTitle', 'ownedDomain3', 'scoreOwnedDomain3',
       'scoreOwnedDomain', 'scoreOwnedDomain2', 'groupScoreOwned', 'nbSearch',
       'facebook', 'linkedIn', 'isFacebook', 'isLinkedIn'],
      dtype='object')

In [31]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


(ts[["denomination", 'scoreOwnedDomain',"statusDomain","ownedDomain" ,"ownedDomain2", "ownedDomain3"]])

,denomination,scoreOwnedDomain,statusDomain,ownedDomain,ownedDomain2,ownedDomain3
0,PROVOST FRERES,0.562500,scierie-provost.fr,scierie-provost.fr,scierie-provost.fr,NONE
1,LA BRIOUXAISE DES BOIS,0.000000,NONE,NONE,NONE,NONE
2,SCIERIE FOREY,0.827586,scierie-forey.fr,scierie-forey.fr,scierie-forey.fr,NONE
3,SCIERIE BARTHELEMY FRERES,0.739130,scieriebarthelemy.com,scieriebarthelemy.com,scieriebarthelemy.com,scieriebarthelemy.com
4,SCIERIE EYMARD SA,0.307692,scierie-construction-bois-isere.com,scierie-construction-bois-isere.com,scierie-construction-bois-isere.com,scierie-construction-bois-isere.com
5,SOCIETE FORESTIERE DU DAUPHINE,0.357143,scierie-mocellin-sofoda.fr,scierie-mocellin-sofoda.fr,scierie-mocellin-sofoda.fr,NONE
6,SOC SA MERCIER,0.500000,mercierbois.fr,mercierbois.fr,mercierbois.fr,NONE
7,SCIERIE SOURD SARL,0.000000,NONE,NONE,NONE,NONE
8,SCIERIE VIRGILIO MOCELLIN ET FILS,0.542373,scierie-mocellin-sofoda.fr,scierie-mocellin-sofoda.fr,scierie-mocellin-sofoda.fr,NONE
9,SCIERIE NIER,0.451613,nier-scierie-38.com,nier-scierie-38.com,nier-scierie-38.com,nier-scierie-38.com


In [35]:
ts["statusDomain"].tolist

<bound method IndexOpsMixin.tolist of 0                             scierie-provost.fr
1                                           NONE
2                               scierie-forey.fr
3                          scieriebarthelemy.com
4            scierie-construction-bois-isere.com
5                     scierie-mocellin-sofoda.fr
6                                 mercierbois.fr
7                                           NONE
8                     scierie-mocellin-sofoda.fr
9                            nier-scierie-38.com
10                                girod-bois.com
11                                          NONE
12                                 lapassade.com
13                           scierie-vuitton.com
14                                          NONE
15                                          NONE
16                                          NONE
17                                  tarteret.com
18                         scieriedugroschene.fr
19                             

In [28]:
def statusDomain(row):
    d=[row['ownedDomain'], row['ownedDomain2'], row['ownedDomain3']]
    dnonone=list(filter(lambda a: a !="NONE", d))
    nb=len(list(dict.fromkeys(d)))
    nbnonone=len(list(dict.fromkeys(dnonone)))
    if nbnonone ==0:
        return "NONE"
    best=row['ownedDomain']
    bestScore=row['scoreOwnedDomain']
    if row['scoreOwnedDomain2'] > bestScore:
        best=row['ownedDomain2']
        bestScore=row['scoreOwnedDomain2']
    
    if row['scoreOwnedDomain3'] > bestScore:
        best=row['ownedDomain3']
        bestScore=row['scoreOwnedDomain3']
    return best

'''d=[
        [row['ownedDomain'],row['scoreOwnedDomain']],
        [row['ownedDomain2'],row['scoreOwnedDomain2']],
        [row['ownedDomain3'],row['scoreOwnedDomain3']]
      ]
'''
ts['statusDomain'] = ts.apply(statusDomain, axis=1)

ts.groupby(['statusDomain'])[['siret']].count()

,siret
statusDomain,
.,1
123devis.com,1
52wmb.com,1
NONE,280
a2bc.eu,1
acronymsandslang.com,1
acteurdurable.org,1
agences.cc,1
ageorges-avis-clients.fr,1


In [ ]:
len(ts[ts['searchTitle'] == '-'])

In [ ]:
ts.groupby(['effectif'])[['siret']].count()

In [ ]:
import pandas as pd
nafs=pd.read_csv("nafs.csv",sep=";")


In [ ]:
ts_naf=pd.merge (ts,nafs)
ts_naf.columns

In [ ]:
ts_naf.groupby(['departement','INTITULÉS 732'])[['siret']].count()

In [ ]:
import pandas as pd

finance=pd.read_csv("chiffres-cles-2019_38.csv",sep=";")
finance.rename(columns={'Siren':'siren'
                                }, inplace=True)
finance = finance.astype({"siren":  int})
ts_naf = ts_naf.astype({"siren":  int})
finance.columns

In [ ]:
ts_naf_fi=pd.merge(ts_naf, finance[['siren','CA 1','CA 2', 'CA 3']], on='siren', how='left')


In [ ]:
ts_naf_fi

In [ ]:
import numpy as np

pd.pivot_table(ts_naf_fi,index=['INTITULÉS 732'],columns=['departement'],values=["CA 1", "CA 2", "CA 3"],aggfunc=[len, np.mean, np.sum]) .style.format('{0:,.0f}')

In [ ]:
pd.pivot_table(ts_naf_fi,index=['INTITULÉS 732'],columns=['departement'],values=["siret"],aggfunc=[len] ,margins=True)

In [ ]:
ts_naf_fi.isnull().sum(axis = 0)


In [ ]:
ts_naf_fi[ts_naf_fi.isna().any(axis=1)]


In [ ]:
from googlesearch import search
search("BOIS DU DAUPHINE")


import lxml.html
t = lxml.html.parse("https://www.dnb.com/business-directory/company-profiles.provost_freres.66acf47c517a112c017d56760ecf141f.html")
print (t.find(".//title").text)

In [ ]:
# importing the modules 
import requests 
from bs4 import BeautifulSoup 
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}

# target url 
url2 = "https://www.dnb.com/business-directory/company-profiles.provost_freres.66acf47c517a112c017d56760ecf141f.html"
url = "https://aurora-5r.fr"
print("kkk")

# making requests instance 
reqs = requests.get(url3,  headers=header) 
print("ddd")
# using the BeaitifulSoup module 
soup = BeautifulSoup(reqs.text, 'html.parser') 
  
# displaying the title 
print("Title of the website is : ") 
for title in soup.find_all('title'): 
    print(title.get_text())

In [ ]:
from urllib.request import urlopen, Request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}


reg_url = url3
req = Request(url=reg_url, headers=headers) 
html = urlopen(req).read() 
print(html)

In [ ]:

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
html = urllib.request.urlopen(url2, context=ctx).read()
html

In [ ]:
url3="https://www.scieriesanguinet.fr/"

def url_status(url):
       headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0)'
                                ' Gecko/20100101 Firefox/24.0'}
       REQUEST_TIMEOUT = 10

       if 'http' not in url:
           url = 'http://' + url
       try:
           response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
           if(response.status_code != 200):
               print(url)
               print('status',response.status_code)
           return response.status_code
       except Exception as e:
           print(url)
           print('Error',e)
           return -1
url_status(url3)

In [ ]:
import requests
import eventlet
eventlet.monkey_patch()

with eventlet.Timeout(10):
    requests.get("http://ipv4.download.thinkbroadband.com/1GB.zip", verify=False)